In [285]:
import numpy as np
from sklearn import preprocessing
import os.path
import librosa
import scipy.stats as stats

#### **2. Extracção de Features.**
2.1. Processar as features do ficheiro top100_features.csv.

2.1.1. Ler o ficheiro e criar um array numpy com as features disponibilizadas.

In [286]:
def extract_features():
    data = np.genfromtxt("Features/top100_features.csv", delimiter = ",", skip_header = 1)
    data = data[:, 1:-1]
    return data


2.1.2. Normalizar as features no intervalo [0, 1].

In [287]:
def normalize_features(data):
    print("data: ", np.shape(data))

    #[[3, 3, [8, 8, 8,8], [9,9,9,9], 7, 7, 7], [3, 3, [8, 8, 8,8], [9,9,9,9], 7, 7, 7]]
    #[3, 3, 8, 8, 8, 8,]
    max_values = data.max(axis = 0)
    min_values = data.min(axis = 0)
    print("max values shape: ", np.shape(max_values))
    print("min values shape: ", np.shape(min_values))

    return (data-min_values)/(max_values - min_values)

2.1.3. Criar e gravar em ficheiro um array numpy com as features extraídas (linhas = músicas; colunas = valores das features).

In [288]:
def save_top100_features_normalized(path, data_normalized):
    np.savetxt(path, data_normalized, delimiter=",", fmt="%.6f")

2.2. Extrair features da framework librosa. <br>

2.2.1. Para os 900 ficheiros da BD, extrair as seguintes features (sugestão: guardar
todas as músicas na mesma pasta):
- Features Espectrais: mfcc, spectral centroid, spectral bandwidth, spectral
contrast, spectral flatness e spectral rolloff.
- Features Temporais: F0, rms e zero crossing rate.
- Outras features: tempo.
- Utilize os parâmetros por omissão do librosa (sr = 22050 Hz, mono, window
length = frame length = 92.88 ms e hop length = 23.22 ms).
- Guarde as features num array numpy 2D, com número de linhas = número de
músicas e número de colunas = número de features

In [289]:
def extract_music_features(music, music_folder):
    path = "Music/"
    
    data, sampling_rate = librosa.load(path + music)

    mfcc = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc = 13)
    spectral_centroid = librosa.feature.spectral_centroid(y=data, sr=sampling_rate)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=data, sr=sampling_rate)
    spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=sampling_rate)
    spectral_flatness = librosa.feature.spectral_flatness(y=data)
    spectral_rolloff = librosa.feature.spectral_rolloff(y = data, sr = sampling_rate)
    f0 = librosa.yin(y=data, fmin=20, fmax=11025)
    rms = librosa.feature.rms(y=data)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=data)
    tempo = librosa.beat.tempo(y=data, sr=sampling_rate)

    print("FEATURES TEMPORAIS")

    #print("mfcc: ", mfcc)
    print("mfcc shape: ", np.shape(mfcc))     

    #print("spectral centroid: ", spectral_centroid)   
    print("spectral centroid shape: ", np.shape(spectral_centroid))     

    #print("spectral bandwidth: ", spectral_bandwidth)
    print("spectral bandwidth shape: ", np.shape(spectral_bandwidth))

    #print("spectral contrast: ", spectral_contrast)
    print("spectral contrast shape: ", np.shape(spectral_contrast))

    #print("spectral flatness: ", spectral_flatness)
    print("spectral flatness shape: ", np.shape(spectral_flatness))     

    #print("spectral rolloff: ", spectral_rolloff)
    print("spectral rolloff shape: ", np.shape(spectral_rolloff))     

    
    print("FEATURES TEMPORAIS")
    #print("f0: ", f0)
    print("f0 shape: ", np.shape(f0))     

    #print("rms: ", rms)
    print("rms shape: ", np.shape(rms))     

    #print("zero crossing rate: ", zero_crossing_rate)
    print("zero crossing rate shape: ", np.shape(zero_crossing_rate))

    #print("tempo: ", tempo)
    print("tempo shape: ", np.shape(tempo))
        
    return np.array([mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_flatness,
                        spectral_rolloff, f0, rms, zero_crossing_rate, tempo])

2.2.2. Calcular as 7 estatísticas típicas sobre as features anteriores: média, desvio
padrão, assimetria (skewness), curtose (kurtosis), mediana, máximo e mínimo.
Para o efeito, utilizar a biblioteca scipy.stats (e.g., scipy.stats.skew).

In [290]:
def calculate_statistics(song):
    
    song_statistics = np.empty((0, 7))
    i = 0
    #[mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectra.....]
    print("song shape: ", np.shape(song))

    axis_ = 1
    for feature in song:
        print("feature shape: ", np.shape(feature))
        print("feature ndim: ", len(feature.shape))

        feat_mean = feature.mean(axis = axis_)
        print("mean: ", feat_mean)
        print("feat mean shape: ", np.shape(feat_mean))
        
        feat_std = feature.std(axis = axis_)

        print("feat std shape: ", np.shape(feat_std))

        feat_skewness = stats.skew(feature, axis = axis_)
        print("feat skewness shape: ", np.shape(feat_skewness))

        feat_kurtosis = stats.kurtosis(feature, axis = axis_)
        print("feat kurtosis shape: ", np.shape(feat_kurtosis))

        feat_median = np.median(feature, axis = axis_)
        print("feat median shape: ", np.shape(feat_median))

        feat_min = feature.min(axis = axis_)
        
        print("feat mean shape: ", np.shape(feat_mean))

        feat_max = feature.max(axis = axis_)
        print("feat max shape: ", np.shape(feat_max))
        
        '''
        
        feature_statistics = np.array([feat_mean, feat_std, feat_skewness, feat_kurtosis, feat_median, feat_min, feat_max])
        print(f" {i} : {np.shape(feature_statistics)}")
        song_statistics = np.vstack([song_statistics, feature_statistics])
        i += 1
        '''
        
    #print("song statistics shape: ", np.shape(song_statistics.flatten()))
    #return song_statistics.flatten()

2.2.3. Normalizar as features no intervalo [0, 1].

In [291]:
# Chamamos a função normalize_features()

2.2.4. Criar e gravar em ficheiro o array numpy com as features extraídas.

#### **Main function**

In [292]:
if __name__ == "__main__" :
    saved_features = "top100_features_normalized.csv"
    music_folder = os.listdir("Music")
    
    # Ex 2.1
    if os.path.exists(saved_features):
        data_normalized = np.genfromtxt(saved_features, delimiter = ",")
    else:
        data_extracted = extract_features()
        data_normalized = normalize_features(data_extracted)
        save_top100_features_normalized(saved_features, data_normalized)
    
    # Ex 2.2
    statistics = np.empty((0, 70))
    total_normalized_statistics = np.array([])
    for music in music_folder[:2]:
        music_features = extract_music_features(music, music_folder)
        print("------- MUSIC FEATURES SHAPE: ", np.shape(music_features))
        calculate_statistics(music_features)
        #statistics = np.vstack([statistics, calculate_statistics(music_features).flatten()])
        #print("statistics shape: ", np.shape(statistics))
        
    #print(statistics)
    #normalized_statistics = normalize_features(statistics)
    #total_normalized_statistics = total_normalized_statistics.append(normalized_statistics)
        #save_top100_features_normalized(saved_features, data_normalized)
    print(total_normalized_statistics)
    
    

C:\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FEATURES TEMPORAIS
mfcc shape:  (13, 1295)
spectral centroid shape:  (1, 1295)
spectral bandwidth shape:  (1, 1295)
spectral contrast shape:  (7, 1295)
spectral flatness shape:  (1, 1295)
spectral rolloff shape:  (1, 1295)
FEATURES TEMPORAIS
f0 shape:  (1295,)
rms shape:  (1, 1295)
zero crossing rate shape:  (1, 1295)
tempo shape:  (1,)
------- MUSIC FEATURES SHAPE:  (10,)
song shape:  (10,)
feature shape:  (13, 1295)
feature ndim:  2
2222222222222222222222222222222222222222222222222
mean:  [-230.35463    100.47692    -32.28401     28.74003     -9.852089
  -15.665549    -5.184987    -1.6802237  -14.312028    -2.2323027
   -5.9331036    6.4956255   -2.4641428]
feat mean shape:  (13,)
feat std shape:  (13,)
feat skewness shape:  (13,)
feat kurtosis shape:  (13,)
feat median shape:  (13,)
feat mean shape:  (13,)
feat max shape:  (13,)
feature shape:  (1, 1295)
feature ndim:  2
2222222222222222222222222222222222222222222222222
mean:  [2131.44499746]
feat mean shape:  (1,)
feat std shape:  

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25560\3030454561.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_flatness,


FEATURES TEMPORAIS
mfcc shape:  (13, 1257)
spectral centroid shape:  (1, 1257)
spectral bandwidth shape:  (1, 1257)
spectral contrast shape:  (7, 1257)
spectral flatness shape:  (1, 1257)
spectral rolloff shape:  (1, 1257)
FEATURES TEMPORAIS
f0 shape:  (1257,)
rms shape:  (1, 1257)
zero crossing rate shape:  (1, 1257)
tempo shape:  (1,)
------- MUSIC FEATURES SHAPE:  (10,)
song shape:  (10,)
feature shape:  (13, 1257)
feature ndim:  2
2222222222222222222222222222222222222222222222222
mean:  [-58.508366  101.55045   -73.78769    71.2898    -22.013943   25.88694
 -13.503682   27.429174   -6.8228     15.4346285   2.1046522   3.8943129
   5.506888 ]
feat mean shape:  (13,)
feat std shape:  (13,)
feat skewness shape:  (13,)
feat kurtosis shape:  (13,)
feat median shape:  (13,)
feat mean shape:  (13,)
feat max shape:  (13,)
feature shape:  (1, 1257)
feature ndim:  2
2222222222222222222222222222222222222222222222222
mean:  [2484.73959221]
feat mean shape:  (1,)
feat std shape:  (1,)
feat skew